In [1]:
import numpy as np
import pickle
import os

seed = 2023

In [2]:
from nlpsig_networks.scripts.swmhau_network_functions import (
    swmhau_network_hyperparameter_search
)

In [3]:
output_dir = "client_talk_type_output"
if not os.path.isdir(output_dir):
    os.makedirs(output_dir)

## AnnoMI

In [9]:
%run ../load_anno_mi.py

In [10]:
anno_mi.head()

,mi_quality,transcript_id,topic,utterance_id,interlocutor,timestamp,utterance_text,annotator_id,therapist_input_exists,therapist_input_subtype,reflection_exists,reflection_subtype,question_exists,question_subtype,main_therapist_behaviour,client_talk_type,datetime,speaker
0,high,0,reducing alcohol consumption,0,therapist,00:00:13,Thanks for filling it out. We give this form t...,3,False,NaN,False,NaN,True,open,question,NaN,2023-09-13 00:00:13,-1
1,high,0,reducing alcohol consumption,1,client,00:00:24,Sure.,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,neutral,2023-09-13 00:00:24,1
2,high,0,reducing alcohol consumption,2,therapist,00:00:25,"So, let's see. It looks that you put-- You dri...",3,True,information,False,NaN,False,NaN,therapist_input,NaN,2023-09-13 00:00:25,-1
3,high,0,reducing alcohol consumption,3,client,00:00:34,Mm-hmm.,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,neutral,2023-09-13 00:00:34,1
4,high,0,reducing alcohol consumption,4,therapist,00:00:34,-and you usually have three to four drinks whe...,3,True,information,False,NaN,False,NaN,therapist_input,NaN,2023-09-13 00:00:34,-1


In [14]:
with open("../anno_mi_sbert.pkl", "rb") as f:
    sbert_embeddings = pickle.load(f)
    
sbert_embeddings.shape

(9699, 384)

# swmhau Network

## Obtaining path by looking at post history

We can obtain a path by looking at the history of each post. Here we look at the last 10 posts (and pad with vectors of zeros if there are less than 10 posts) including the current post.

We only want to consider paths that correspond to a client's utterance as we want to model a change in mood at that time. Their history will still contain the therapist's utterances too.

In [15]:
features = ["timeline_index", "speaker"]
standardise_method = [None, None]
include_features_in_path = False

In [16]:
num_epochs = 100
dimensions = [15] # [50, 15]
# define swmhau parameters: (output_channels, sig_depth, num_heads)
swmhau_parameters = [(12, 3, 10), (8, 4, 6)]#, (8, 4, 12)]
num_layers = [1]
ffn_hidden_dim_sizes = [[128,128],[256,256],[512,512]]
dropout_rates = [0.1, 0.2]
learning_rates = [1e-3, 1e-4, 5e-4]
seeds = [1, 12, 123]
loss = "focal"
gamma = 2
validation_metric = "f1"
patience = 5

## UMAP

In [17]:
size = 20
swmhau_network_umap_kfold_20, best_swmhau_network_umap_kfold_20, _, __ = swmhau_network_hyperparameter_search(
    num_epochs=num_epochs,
    df=anno_mi,
    id_column="transcript_id",
    label_column="client_talk_type",
    embeddings=sbert_embeddings,
    y_data=y_data_client,
    output_dim=output_dim_client,
    history_lengths=[size],
    dim_reduce_methods=["umap"],
    dimensions=dimensions,
    log_signature=True,
    swmhau_parameters=swmhau_parameters,
    num_layers=num_layers,
    ffn_hidden_dim_sizes=ffn_hidden_dim_sizes,
    dropout_rates=dropout_rates,
    learning_rates=learning_rates,
    seeds=seeds,
    loss=loss,
    gamma=gamma,
    features=features,
    standardise_method=standardise_method,
    include_features_in_path=include_features_in_path,
    path_indices=client_index,
    k_fold=True,
    patience=patience,
    validation_metric=validation_metric,
    results_output=f"{output_dir}/swmhau_network_umap_focal_{gamma}_{size}_kfold.csv",
    verbose=False
)

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]


##################################################
dimension: 15 | method: umap
[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] - columns beginning with 'd' denote the dimension reduced embeddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' feature...
[INFO] Adding 'time_diff' feature...
[INFO] Adding 'timeline_index' feature...
[INFO] Padding ids and storing in `.df_padded` and `.array_padded` attributes.


  0%|          | 0/9699 [00:00<?, ?it/s]

...

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] - columns beginning with 'd' denote the dimension reduced embeddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' feature...
[INFO] Adding 'time_diff' feature...
[INFO] Adding 'timeline_index' feature...
[INFO] Padding ids and storing in `.df_padded` and `.array_padded` attributes.


  0%|          | 0/9699 [00:00<?, ?it/s]

[INFO] The path was created for each item in the dataframe, by looking at its history, so to include embeddings in the FFN input, we concatenate the embeddings for each sentence / text.
saving results dataframe to CSV for this hyperparameter search in client_talk_type_output/swmhau_network_umap_focal_2_20_kfold.csv
saving the best model results dataframe to CSV for this hyperparameter search in client_talk_type_output/swmhau_network_umap_focal_2_20_kfold_best_model.csv


In [18]:
swmhau_network_umap_kfold_20.groupby([
    "dimensions",
    "output_channels",
    "sig_depth",
    "num_heads",
    "num_layers",
    "ffn_hidden_dim",
    "learning_rate"
]).mean()

/tmp/ipykernel_2162024/2207682611.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  swmhau_network_umap_kfold_20.groupby([


accuracy  \
dimensions output_channels sig_depth num_heads num_layers ffn_hidden_dim learning_rate             
15         8               4         6         1          (128, 128)     0.0001         0.604872   
                                                                         0.0005         0.646945   
                                                                         0.0010         0.675455   
                                                          (256, 256)     0.0001         0.616670   
                                                                         0.0005         0.648986   
                                                                         0.0010         0.666148   
                                                          (512, 512)     0.0001         0.611722   
                                                                         0.0005         0.646564   
                                                                         0.0010         0.660127   
           12              3         10        1          (128, 128)     0.0001         0.616047   
                                                                         0.0005         0.626669   
                                                                         0.0010         0.664937   
                                                          (256, 256)     0.0001         0.618781   
                                                                         0.0005         0.621168   
                                                                         0.0010         0.662342   
                                                          (512, 512)     0.0001         0.606532   
                                                                         0.0005         0.630475   
                                                                         0.0010         0.661062   

                                                                                              f1  \
dimensions output_channels sig_depth num_heads num_layers ffn_hidden_dim learning_rate             
15         8               4         6         1          (128, 128)     0.0001         0.404567   
                                                                         0.0005         0.513636   
                                                                         0.0010         0.553468   
                                                          (256, 256)     0.0001         0.394967   
                                                                         0.0005         0.516439   
                                                                         0.0010         0.541551   
                                                          (512, 512)     0.0001         0.409086   
                                                                         0.0005         0.524550   
                                                                         0.0010         0.538057   
           12              3         10        1          (128, 128)     0.0001         0.439761   
                                                                         0.0005         0.473978   
                                                                         0.0010         0.546869   
                                                          (256, 256)     0.0001         0.440443   
                                                                         0.0005         0.479395   
                                                                         0.0010         0.542998   
                                                          (512, 512)     0.0001         0.453031   
                                                                         0.0005         0.488697   
                                                                         0.0010         0.538134   

                                                                                      

In [19]:
best_swmhau_network_umap_kfold_20

,loss,accuracy,f1,f1_scores,precision,precision_scores,recall,recall_scores,valid_loss,valid_accuracy,...,learning_rate,seed,loss_function,gamma,k_fold,n_splits,augmentation_type,hidden_dim_aug,comb_method,batch_size
0,None,0.680507,0.551493,"[0.7960588051298093, 0.4921135646687697, 0.366...",0.561693,"[0.7726168791742563, 0.5244956772334294, 0.387...",0.543801,"[0.8209677419354838, 0.4634974533106961, 0.346...",None,0.671069,...,0.001,1,focal,2,True,5,Conv1d,None,concatenation,64
0,None,0.669296,0.549062,"[0.7877156878264998, 0.480188679245283, 0.3792...",0.552905,"[0.7733913584084551, 0.5403397027600849, 0.344...",0.551940,"[0.8025806451612904, 0.43208828522920206, 0.42...",None,0.670440,...,0.001,12,focal,2,True,5,Conv1d,None,concatenation,64
0,None,0.673448,0.552236,"[0.7928101428342159, 0.4969591659426586, 0.366...",0.552404,"[0.7888853401469179, 0.5088967971530249, 0.359...",0.552370,"[0.7967741935483871, 0.4855687606112054, 0.374...",None,0.667453,...,0.001,123,focal,2,True,5,Conv1d,None,concatenation,64


In [20]:
best_swmhau_network_umap_kfold_20["f1"].mean()

0.5509304745200074

In [21]:
best_swmhau_network_umap_kfold_20["precision"].mean()

0.5556675519188042

In [22]:
best_swmhau_network_umap_kfold_20["recall"].mean()

0.549370469183712

In [23]:
np.stack(best_swmhau_network_umap_kfold_20["f1_scores"]).mean(axis=0)

array([0.79219488, 0.4897538 , 0.37084274])

In [24]:
np.stack(best_swmhau_network_umap_kfold_20["precision_scores"]).mean(axis=0)

array([0.77829786, 0.52457739, 0.3641274 ])

In [25]:
np.stack(best_swmhau_network_umap_kfold_20["recall_scores"]).mean(axis=0)

array([0.80677419, 0.46038483, 0.38095238])

## Random Projections

In [26]:
size = 20
swmhau_network_grp_kfold_20, best_swmhau_network_grp_kfold_20, _, __ = swmhau_network_hyperparameter_search(
    num_epochs=num_epochs,
    df=anno_mi,
    id_column="transcript_id",
    label_column="client_talk_type",
    embeddings=sbert_embeddings,
    y_data=y_data_client,
    output_dim=output_dim_client,
    history_lengths=[size],
    dim_reduce_methods=["gaussian_random_projection"],
    dimensions=dimensions,
    log_signature=True,
    swmhau_parameters=swmhau_parameters,
    num_layers=num_layers,
    ffn_hidden_dim_sizes=ffn_hidden_dim_sizes,
    dropout_rates=dropout_rates,
    learning_rates=learning_rates,
    seeds=seeds,
    loss=loss,
    gamma=gamma,
    features=features,
    standardise_method=standardise_method,
    include_features_in_path=include_features_in_path,
    path_indices=client_index,
    k_fold=True,
    patience=patience,
    validation_metric=validation_metric,
    results_output=f"{output_dir}/swmhau_network_grp_focal_{gamma}_{size}_kfold.csv",
    verbose=False
)

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]


##################################################
dimension: 15 | method: gaussian_random_projection
[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] - columns beginning with 'd' denote the dimension reduced embeddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' feature...
[INFO] Adding 'time_diff' feature...
[INFO] Adding 'timeline_index' feature...
[INFO] Padding ids and storing in `.df_padded` and `.array_padded` attributes.


  0%|          | 0/9699 [00:00<?, ?it/s]

...

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] - columns beginning with 'd' denote the dimension reduced embeddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' feature...
[INFO] Adding 'time_diff' feature...
[INFO] Adding 'timeline_index' feature...
[INFO] Padding ids and storing in `.df_padded` and `.array_padded` attributes.


  0%|          | 0/9699 [00:00<?, ?it/s]

[INFO] The path was created for each item in the dataframe, by looking at its history, so to include embeddings in the FFN input, we concatenate the embeddings for each sentence / text.
saving results dataframe to CSV for this hyperparameter search in client_talk_type_output/swmhau_network_grp_focal_2_20_kfold.csv
saving the best model results dataframe to CSV for this hyperparameter search in client_talk_type_output/swmhau_network_grp_focal_2_20_kfold_best_model.csv


In [27]:
swmhau_network_grp_kfold_20.groupby([
    "dimensions",
    "output_channels",
    "sig_depth",
    "num_heads",
    "num_layers",
    "ffn_hidden_dim",
    "dropout_rate",
    "learning_rate"
]).mean()

/tmp/ipykernel_2162024/2744579754.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  swmhau_network_grp_kfold_20.groupby([


accuracy  \
dimensions output_channels sig_depth num_heads num_layers ffn_hidden_dim dropout_rate learning_rate             
15         8               4         6         1          (128, 128)     0.1          0.0001         0.601135   
                                                                                      0.0005         0.597675   
                                                                                      0.0010         0.638987   
                                                                         0.2          0.0001         0.602796   
                                                                                      0.0005         0.616843   
                                                                                      0.0010         0.652758   
                                                          (256, 256)     0.1          0.0001         0.619473   
                                                                                      0.0005         0.602450   
                                                                                      0.0010         0.650958   
                                                                         0.2          0.0001         0.612968   
                                                                                      0.0005         0.599820   
                                                                                      0.0010         0.653588   
                                                          (512, 512)     0.1          0.0001         0.600720   
                                                                                      0.0005         0.603280   
                                                                                      0.0010         0.653242   
                                                                         0.2          0.0001         0.588610   
                                                                                      0.0005         0.608954   
                                                                                      0.0010         0.646668   
           12              3         10        1          (128, 128)     0.1          0.0001         0.619542   
                                                                                      0.0005         0.615598   
                                                                                      0.0010         0.615874   
                                                                         0.2          0.0001         0.624801   
                                                                                      0.0005         0.621618   
                                                                                      0.0010         0.623901   
                                                          (256, 256)     0.1          0.0001         0.615528   
                                                                                      0.0005         0.611515   
                                                                                      0.0010         0.623832   
                                                                         0.2          0.0001         0.617051   
                                                                                      0.0005         0.597121   
                                                                                      0.0010         0.617051   
                                                          (512, 512)     0.1          0.0001         0.626808   
                                                                                      0.0005         0.606117   
                                                                                      0.0010         0.615390   
                                                                         0.2          0.0001         0.600650   
                                 

In [28]:
best_swmhau_network_grp_kfold_20

,loss,accuracy,f1,f1_scores,precision,precision_scores,recall,recall_scores,valid_loss,valid_accuracy,...,learning_rate,seed,loss_function,gamma,k_fold,n_splits,augmentation_type,hidden_dim_aug,comb_method,batch_size
0,None,0.659954,0.539945,"[0.782135425004002, 0.4668741662961316, 0.3708...",0.539582,"[0.7762948840165237, 0.49019607843137253, 0.35...",0.541734,"[0.7880645161290323, 0.4456706281833616, 0.391...",None,0.659277,...,0.001,1,focal,2,True,5,Conv1d,None,concatenation,64
0,None,0.659124,0.526223,"[0.7856695869837297, 0.4387947269303202, 0.354...",0.532443,"[0.7627582017010935, 0.492600422832981, 0.3419...",0.524311,"[0.81, 0.39558573853989815, 0.3673469387755102]",None,0.646855,...,0.001,12,focal,2,True,5,Conv1d,None,concatenation,64
0,None,0.641686,0.532949,"[0.7632450331125828, 0.46935483870967737, 0.36...",0.528600,"[0.7840136054421769, 0.4470046082949309, 0.354...",0.538695,"[0.7435483870967742, 0.4940577249575552, 0.378...",None,0.636321,...,0.001,123,focal,2,True,5,Conv1d,None,concatenation,64


In [29]:
best_swmhau_network_grp_kfold_20["f1"].mean()

0.5330390532239303

In [30]:
best_swmhau_network_grp_kfold_20["precision"].mean()

0.533541453065807

In [31]:
best_swmhau_network_grp_kfold_20["recall"].mean()

0.5349131416727827

In [32]:
np.stack(best_swmhau_network_grp_kfold_20["f1_scores"]).mean(axis=0)

array([0.77701668, 0.45834124, 0.36375923])

In [33]:
np.stack(best_swmhau_network_grp_kfold_20["precision_scores"]).mean(axis=0)

array([0.77435556, 0.47660037, 0.34966843])

In [34]:
np.stack(best_swmhau_network_grp_kfold_20["recall_scores"]).mean(axis=0)

array([0.78053763, 0.4451047 , 0.37909709])